# App that calculates the price of a printed part.


In [36]:
#from dotenv import load_dotenv
import os  
from dotenv import load_dotenv
load_dotenv(override=True) 



# High level definitions of costs and constants
usd_conv = 1300  # $ / usd

MARGIN = float(os.environ['MARGIN_PERCENTAGE_BEFORE_TAXES']) # part of the cost (before taxes) that is pure gain

# Material related costs
material_unit_price = 19  # usd / kg
# here we could add some consumable as platform sheet / hotend replacement / bowden tube deterioration

# Printer related costs
PRINTER_COST                    = float(os.environ['PRINTER_COST_IN_USD'])
PRINTER_LIFETIME_IN_YEARS       = float(os.environ['PRINTER_WORKING_LIFETIME_IN_YEARS'])
printer_lifetime_in_min         = PRINTER_LIFETIME_IN_YEARS * 365.0 * 24 * 60
success_rate                    = 0.8  # it represents the % of times the print results in success
lifetime_working_min            = printer_lifetime_in_min * success_rate
printing_time_cost_by_min       = PRINTER_COST / lifetime_working_min  # usd / min

# Personnel costs:
SLICER_TECHNICIAN_HOURLY_RATE   = float(os.environ['TECHNICIAN_HOURLY_RATE'])
ENGINEER_HOURLY_RATE            = float(os.environ['ENGINEER_HOURLY_RATE'])

In [43]:
# Sliced part attributes:

printing_time = float( 9*60 ) # min 500gr  y 11.5*60 min
material_gr         = float( 600 )  # gr

material_difficulty_factor = 1 # for materials that may have issues printing or require special care
postprocessing_support = 'False' 

slicing_time        = 40 #+ 10*printing_time/60   # min
engineering_time    = 10   # min

In [46]:

# to estimate the printing time cost a simple investement calculation is done 

employees_cost              = slicing_time  * SLICER_TECHNICIAN_HOURLY_RATE / 60    +  engineering_time * ENGINEER_HOURLY_RATE / 60
equipment_depreciation_cost = printing_time * printing_time_cost_by_min
material_cost               = material_gr   * material_unit_price / 1000  
#--------------------------------------------------------------------------------------------
total_cost                  = employees_cost + equipment_depreciation_cost + material_cost
price                       = total_cost/(1-MARGIN/100.0)


In [47]:
# Print the calculations

print(
    f''' Cost: $ {"%.0f" % total_cost} \t $Arg {"%.0f" % (total_cost*usd_conv)}''')
print(
    f''' Price: $ {"%.0f" % total_cost} \t $Arg {"%.0f" % (price*usd_conv)}''')

print(" Partial costs: ")
print(
    f''' - Employees: \t {"%.0f" % ( employees_cost*100/total_cost)}%  \t USD {employees_cost} \t $Arg {employees_cost*usd_conv}''')
print(f''' - Equip_depx: \t {"%.0f" % ( equipment_depreciation_cost*100/total_cost)}%  \t USD {"%.1f" % equipment_depreciation_cost} \t $Arg {"%.1f" % ( equipment_depreciation_cost*usd_conv)}''')
print(
    f''' - Material:  \t {"%.0f" % ( material_cost*100/total_cost)}%  \t USD {material_cost} \t $Arg {material_cost*usd_conv}''')

 Cost: $ 35 	 $Arg 45829
 Price: $ 35 	 $Arg 65469
 Partial costs: 
 - Employees: 	 57%  	 USD 20.0 	 $Arg 26000.0
 - Equip_depx: 	 11%  	 USD 3.9 	 $Arg 5008.6
 - Material:  	 32%  	 USD 11.4 	 $Arg 14820.0
